In [1]:
import os, sys, glob, time

import numpy as np
import pandas as pd

import math

# Parallel proces
from myparproc.ParProc import ParProc
from myparproc.ProgressCheck import ProgressCheck

# Linear algebra
import myutil.myutil as util

# Plotting tools
import matplotlib.pyplot as plt

import rospy, rosbag

# For logging to csv
import csv

from livox_ros_driver.msg import CustomMsg as lvCloud
from livox_ros_driver.msg import CustomPoint as lvPoint

# For kd tree search
# import pcl, pcl_ros, pcl_msgs

# For load/unloading pcd/ply
import pypcd
from pypcd import pypcd

# For loading/unloading pcd/ply
# from pyntcloud import PyntCloud

# Some processing or plotting
import open3d as o3d

from ceva import Ceva

# Lidar topic
lidar_topic = '/livox/lidar'

# Location of the dataset
bag_file = '/media/tmn/mySataSSD1/DATASETS/MCDVIRAL/PublishedSequencesUnzipped/ntu_day_10/ntu_day_10_mid70.bag'

# Output location
output_dir = '/media/tmn/mySataSSD1/Experiments/ctgaupro/mcd_ntu_day_10_lidars/'
os.makedirs(output_dir + lidar_topic.replace('/', '__'), exist_ok=True)

# Fields to export to ply
interested_fields = ['x', 'y', 'z', 'intensity', 't']


topicmanager initialized
Log opened: Wed Sep 11 05:29:10 2024 UTC


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:

def lvCloudToArray(msg, cloud_idx):
    x = []
    y = []
    z = []
    i = []
    t = []
    
    for lvP in msg.points:
        x.append(lvP.x)
        y.append(lvP.y)
        z.append(lvP.z)
        i.append(lvP.reflectivity)
        t.append(lvP.offset_time)

    points = list(zip(x, y, z, i, t))
    points = np.array(points, dtype=[('x',         'f4'),
                                     ('y',         'f4'),
                                     ('z',         'f4'),
                                     ('intensity', 'f4'),
                                     ('t',         'i4'),
                                    ])

    return points

# Load the pointcloud in the bag
bag = rosbag.Bag(bag_file)

# Get total message count
total_msg = bag.get_message_count(lidar_topic)
count_max_digit = len(str(total_msg))
print("Total msg: ", total_msg, count_max_digit)

progress = 0
count = -1
for topic, msg, t in bag.read_messages():
    if topic == lidar_topic:
    
        # Increment counter
        count += 1

        # Timestamp
        timestamp = msg.header.stamp.to_sec()

        # Extract the point cloud from message
        lvpc_in = lvCloudToArray(msg, count)
        
        # Sort points by sampling time, then ring (lexsort uses the reverse order)
        lvpc_in = lvpc_in[np.argsort(lvpc_in, order='t')]
       
        # Make the file name
        lvpc_filename = output_dir + '/' + lidar_topic.replace('/', '__') + '/' + f'{timestamp:.9f}' + '.pcd'
        
        # Save the pointcloud
        pypcd.save_point_cloud_bin_compressed(pypcd.PointCloud.from_array(lvpc_in), lvpc_filename)
        
        # Report the progress
        if (math.floor(count/total_msg*100) != progress or count == total_msg - 1):
            progress = math.floor(count/total_msg*100)
            print(f'Export {progress:3d}%')
        # print(f'Seq: {seq}. Cloud {count}. SweepTime: {sweepTime}')

# Need to close the log file to get the text out
bag.close()

Total msg:  3247 4
Export   1%
Export   2%
Export   3%
Export   4%
Export   5%
Export   6%
Export   7%
Export   8%
Export   9%
Export  10%
Export  11%
Export  12%
Export  13%
Export  14%
Export  15%
Export  16%
Export  17%
Export  18%
Export  19%
Export  20%
Export  21%
Export  22%
Export  23%
Export  24%
Export  25%
Export  26%
Export  27%
Export  28%
Export  29%
Export  30%
Export  31%
Export  32%
Export  33%
Export  34%
Export  35%
Export  36%
Export  37%
Export  38%
Export  39%
Export  40%
Export  41%
Export  42%
Export  43%
Export  44%
Export  45%
Export  46%
Export  47%
Export  48%
Export  49%
Export  50%
Export  51%
Export  52%
Export  53%
Export  54%
Export  55%
Export  56%
Export  57%
Export  58%
Export  59%
Export  60%
Export  61%
Export  62%
Export  63%
Export  64%
Export  65%
Export  66%
Export  67%
Export  68%
Export  69%
Export  70%
Export  71%
Export  72%
Export  73%
Export  74%
Export  75%
Export  76%
Export  77%
Export  78%
Export  79%
Export  80%
Export  81%
Export  8